In [1]:
import requests
import getpass
cid = getpass.getpass()
secret_id = getpass.getpass()

········
········


In [2]:
# !pip install spotipy

In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
from pandas import json_normalize

In [4]:
secrets_file = open('secrets.txt','r')
secrets_file

<_io.TextIOWrapper name='secrets.txt' mode='r' encoding='UTF-8'>

In [5]:
string = secrets_file.read()
string

'client_id:d2b0e15338ce4373808898efbef3738c\nclient_secret:12f3e85b1b9c4aa2b1e8f586601caf9e'

In [6]:
string.split('\n')

['client_id:d2b0e15338ce4373808898efbef3738c',
 'client_secret:12f3e85b1b9c4aa2b1e8f586601caf9e']

In [7]:
secrets_dict={}
for line in string.split('\n'):
    if len(line)>0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [8]:
secrets_dict

{'client_id': 'd2b0e15338ce4373808898efbef3738c',
 'client_secret': '12f3e85b1b9c4aa2b1e8f586601caf9e'}

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['client_id'],
                                                           client_secret=secrets_dict['client_secret']))

In [10]:
playlist = sp.user_playlist_tracks("spotify", "6yPiKpy7evrwvZodByKvM9")

In [11]:
playlist["total"]

10000

In [12]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        try:
            results = sp.next(results)
            tracks = tracks + results['items']
            sleep(randint(1,3))
        except:
            print('\not there.')
            time.sleep(5)
    return tracks

In [ ]:
all_tracks = get_playlist_tracks("6yPiKpy7evrwvZodByKvM9")

In [ ]:
all_tracks[0]['track']['artists'][0]['name']

In [ ]:

all_tracks[0]['track']['external_urls']['spotify']

In [ ]:
all_tracks[0]["track"]["name"]

In [ ]:
all_tracks[0]['track']['uri']

In [ ]:
all_tracks[1]['track']['popularity']

In [ ]:

df = pd.DataFrame(columns=['artist','song','popularity', 'uri', 'url'])


for track in all_tracks:
    artist_name = track['track']['artists'][0]['name']
    song_name = track['track']['name']
    popularity = track["track"]["popularity"]
    uri = track["track"]["uri"]
    url = track["track"]["external_urls"]
    df = df.append({'artist':artist_name,'song':song_name, 'popularity':popularity, 'uri':uri, 'url':url}, ignore_index=True)

print(df)

In [ ]:

# sp.audio_features(song_uri)

In [ ]:
df['url'] = json_normalize(df['url'])

In [ ]:
df.to_csv('df.csv', index=False)

In [ ]:
# features
       

In [ ]:
df.head()

In [ ]:
song_uri=df['uri'][0]

In [ ]:
sp.audio_features(song_uri)

In [ ]:
df.shape

In [ ]:
audio_features = []
for uri in df["uri"]:
    audio_features.append(sp.audio_features(uri))

In [ ]:
len(audio_features)

In [ ]:
df_features = [pd.DataFrame(d, index=[i]) for i, d in enumerate(audio_features)]
df_features_full = pd.concat(df_features)

In [ ]:
df_data = pd.merge(left = df,
                                  right = df_features_full,
                                  how = 'inner', 
                                  left_on = "uri", 
                                  right_on= "uri")
df_data

In [ ]:
df_data.to_csv("df_data.csv", index = False)

In [ ]:
df_data = pd.read_csv('df_data.csv')
df_data